In [2]:
%pip install optuna==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
dataset_path = "/content/drive/MyDrive/Maestria/Especializacion/DM Economia y Finanzas"
dataset_file = '/competencia_01.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202102
mes_test = 202104

# agregue sus semillas
semilla = 35

data = pd.read_csv(dataset_path + dataset_file)

In [6]:
data = data.drop(columns=['mprestamos_personales'])


In [8]:
data.shape

(981946, 154)

In [9]:
data.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249239632,202101,1,0,0,52,263,12.90,-1020.41,951.02,...,4.0,0.0,-57325.80,0.0,5785.0,13134.60,7.0,0.0,2744.82,CONTINUA
1,249239632,202102,1,0,0,52,264,849.59,617.02,1862.72,...,-3.0,0.0,-28492.69,0.0,5813.0,16111.71,9.0,0.0,3448.62,CONTINUA
2,249239632,202103,1,0,0,52,265,1954.04,3460.90,898.96,...,1.0,0.0,-35025.01,0.0,5844.0,11427.81,5.0,0.0,3131.91,CONTINUA
3,249239632,202104,1,0,0,52,266,2610.89,7231.49,1619.99,...,2.0,0.0,-29334.20,0.0,5874.0,18525.47,8.0,0.0,3835.71,CONTINUA
4,249239632,202105,1,0,0,52,267,2963.71,10969.09,1121.34,...,5.0,0.0,-37566.22,0.0,5905.0,10756.25,4.0,0.0,3624.57,BAJA+2


Vamos a asignar pesos a las clases. En unos minutos explicaremos las razones detrás de esta decisión. Mientras tanto, pueden aprovechar el código para ajustar el peso de la clase **BAJA+2** según lo deseen.





clase_binaria2: toma el valor 1 si clase_ternaria es diferente de 'CONTINUA'

In [10]:
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00003
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [11]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [12]:
train_data = data[data['foto_mes'] == mes_train]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [13]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
Xif = imp_mean.fit_transform(X_test)

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases BAJA+1 y BAJA+2 en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el weight. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.

In [14]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00003, ganancia_acierto, 0) - np.where(weight < 1.00003, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

LGBM necesita su propio tipo de Datasets:


In [15]:
train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

In [16]:
def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 8, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 1000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    lambda_l1 = trial.suggest_float('lambda_l1', 0.0, 10.0)
    lambda_l2 = trial.suggest_float('lambda_l2', 0.0, 10.0)
    min_gain_to_split = trial.suggest_float('min_gain_to_split', 0.0, 1.0)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 7)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': max_depth,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'bagging_freq': bagging_freq,
        'seed': semilla,
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=100,
        # early_stopping_rounds= int(50 + 5 / learning_rate),
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semilla
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


In [17]:
storage_name = "sqlite:///" + dataset_path + "optimization_lgbm.db"
study_name = "exp_302_lgbm"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-10-13 23:00:23,544] Using an existing study with name 'exp_302_lgbm' instead of creating a new one.


[I 2024-10-12 16:58:27,001] Trial 77 finished with value: 114506000.0 and parameters: {'num_leaves': 24, 'learning_rate': 0.11681200404658175, 'min_data_in_leaf': 200, 'feature_fraction': 0.23784197896792073, 'bagging_fraction': 0.6944512419517778, 'max_depth': 10, 'lambda_l1': 1.1536973623742635, 'lambda_l2': 3.3304940551026623, 'min_gain_to_split': 0.2827894890056999, 'bagging_freq': 2}. Best is trial 77 with value: 114506000.0.


In [30]:
study.optimize(objective, n_trials=550) # subir subir

[I 2024-10-13 23:21:04,345] Trial 378 finished with value: 150682000.0 and parameters: {'num_leaves': 20, 'learning_rate': 0.10774427352292713, 'min_data_in_leaf': 254, 'feature_fraction': 0.2179915080364973, 'bagging_fraction': 0.9643366209192407, 'max_depth': 10, 'lambda_l1': 0.8078567807371961, 'lambda_l2': 9.410628903503525, 'min_gain_to_split': 0.4355862000627593, 'bagging_freq': 5}. Best is trial 378 with value: 150682000.0.
[I 2024-10-13 23:21:54,379] Trial 379 finished with value: 149954000.0 and parameters: {'num_leaves': 20, 'learning_rate': 0.11131845481394964, 'min_data_in_leaf': 248, 'feature_fraction': 0.1610227947476305, 'bagging_fraction': 0.9688318804963005, 'max_depth': 11, 'lambda_l1': 0.8313035393302586, 'lambda_l2': 8.296939031691792, 'min_gain_to_split': 0.4310105053155769, 'bagging_freq': 5}. Best is trial 378 with value: 150682000.0.
[I 2024-10-13 23:22:36,068] Trial 380 finished with value: 148344000.0 and parameters: {'num_leaves': 19, 'learning_rate': 0.26585

KeyboardInterrupt: 

In [31]:
plot_param_importances(study)

[I 2024-10-13 03:49:49,418] Trial 293 finished with value: 114828000.0 and parameters: {'num_leaves': 16, 'learning_rate': 0.0904129007526445, 'min_data_in_leaf': 292, 'feature_fraction': 0.28180962949951294, 'bagging_fraction': 0.9809595314504485, 'max_depth': 19, 'lambda_l1': 1.6196058700397447, 'lambda_l2': 8.275196353225926, 'min_gain_to_split': 0.3894426147286418, 'bagging_freq': 4}. Best is trial 293 with value: 114828000.0.


Y finalmente tomamos el mejor modelo y lo entrenamos con la totalidad de los
datos

In [51]:
mes_train = 202104
mes_test = 202106

In [52]:
train_data = data[data['foto_mes'] == mes_train]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [53]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
Xif = imp_mean.fit_transform(X_test)

In [54]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor modelo: {best_iter}")

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'max_depth': study.best_trial.params['max_depth'],  # Nuevo parámetro
    'lambda_l1': study.best_trial.params['lambda_l1'],  # Nuevo parámetro
    'lambda_l2': study.best_trial.params['lambda_l2'],  # Nuevo parámetro
    'min_gain_to_split': study.best_trial.params['min_gain_to_split'],  # Nuevo parámetro
    'bagging_freq': study.best_trial.params['bagging_freq'],  # Nuevo parámetro
    'seed': semilla,
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)


Mejor cantidad de árboles para el mejor modelo: 94
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [36]:
importances = model.feature_importance()
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0]


,feature,importance
106,ctrx_quarter,119
126,Master_fechaalta,114
22,mcuentas_saldo,102
134,Visa_Fvencimiento,96
141,Visa_mlimitecompra,81
...,...,...
118,Master_mconsumosdolares,1
47,cseguro_vivienda,1
46,cseguro_auto,1
45,cseguro_vida,1


Prediccion de abril

In [55]:
y_pred_lgm = model.predict(X_test)

Estrategia de enviar a los primeros 12000

In [56]:
# Agregar las probabilidades al DataFrame de prueba
test_data['pred_prob'] = y_pred_lgm  # Aquí debes usar las probabilidades predichas

# Ordenar las predicciones por la probabilidad de forma descendente
test_data = test_data.sort_values(by='pred_prob', ascending=False)

# Marcar los primeros X como positivos (ejemplo X = 10000)
X = 14000
test_data['Predicted'] = 0  # Inicializar todos como 0 (negativo)
test_data.iloc[:X, test_data.columns.get_loc('Predicted')] = 1  # Marcar los primeros X como 1 (positivo)

# Crear el archivo de salida para Kaggle
output = test_data[['numero_de_cliente', 'Predicted']]

# Guardar el archivo en formato CSV para Kaggle, usando la variable semilla en el nombre
nombre_archivo = f'predicciones_kaggle_{semilla}.csv'
output.to_csv(nombre_archivo, index=False)
print(f"Archivo {nombre_archivo} guardado.")

# Descargar el archivo a tu computadora
from google.colab import files
files.download(nombre_archivo)

<ipython-input-56-6d9e7d0c3475>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Archivo predicciones_kaggle_35.csv guardado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
def ganancia_prob(y_pred, y_true, prop = 1):
  ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
  return ganancia[y_pred >= 0.025].sum() / prop

#print("Ganancia RF:", ganancia_prob(y_pred_rf, y_test_binaria1))
print("Ganancia LGBM:", ganancia_prob(y_pred_lgm, y_test_binaria1))


Ganancia LGBM: -108409000.0


Estrategia de punto de corte

In [ ]:
# Agregar las probabilidades al DataFrame de prueba
test_data['pred_prob'] = y_pred_lgm  # Aquí debes usar las probabilidades predichas

# Clasificar según un punto de corte (threshold)
threshold = 0.025  # Puedes ajustar este valor según sea necesario
test_data['Predicted'] = (test_data['pred_prob'] >= threshold).astype(int)  # 1 si la probabilidad es mayor o igual al threshold, 0 en caso contrario

# Crear el archivo de salida para Kaggle
output = test_data[['numero_de_cliente', 'Predicted']]

# Guardar el archivo en formato CSV para Kaggle, usando la variable semilla en el nombre
nombre_archivo = f'predicciones_kaggle_{semilla}.csv'
output.to_csv(nombre_archivo, index=False)
print(f"Archivo {nombre_archivo} guardado.")

# Descargar el archivo a tu computadora
from google.colab import files
files.download(nombre_archivo)

Archivo predicciones_kaggle_27.csv guardado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Qué conclusión sacó de comparar las curvas de ganancia entre publica y privada para distintas semillas?

Hay un nuevo pero. Sería fabuloso poder contar con la curva de todo el **leaderboard público**, pero no es posible. Hay una cantidad finita de envíos. El siguiente análisis de back testing es más exacto.

**JUEGUE**
- Ponga en False el parámetro **private**
- Cambie la semilla
- Determine la cantidad de casos que va a enviar al leaderboard público. Recuerde que no tiene más de 20 por día.
- Determine la cantidad optima mirando la gráfica
- Ponga el True poder visualizar la curva del **leaderboard privado**
- Evalue que tan bien le acerto.
- Repita

In [29]:
def analisis_2(semilla, desde, paso, cantidad, private = False):

  df_cut_point = pd.DataFrame({'ganancia': ganancia, 'y_pred_lgm': y_pred_lgm})
  df_cut_point['nro_envios'] = df_cut_point.reset_index().index

  plt.figure(figsize=(10, 6))

  private_idx, public_idx = train_test_split(df_cut_point.index, test_size=0.3, random_state=semilla, stratify=y_test_binaria1)

  df_cut_point['public'] = 0.0
  df_cut_point.loc[public_idx, 'public'] = ganancia[public_idx] / 0.3
  df_cut_point['public_cum'] = df_cut_point['public'].cumsum()

  maximo_paso = desde + paso*cantidad
  plt.plot(df_cut_point['nro_envios'][list(range(desde, maximo_paso + 1, paso))], df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))], label='Ganancia Pública Acumulada')
  max_public_cum = df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))].max()
  max_public_idx = df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))].idxmax()
  plt.axvline(x=max_public_idx, color='g', linestyle='--', label=f'Máximo Ganancia Pública en {max_public_idx}')

  if private:
    df_cut_point['private'] = 0.0
    df_cut_point.loc[private_idx, 'private'] = ganancia[private_idx] / 0.7
    df_cut_point['private_cum'] = df_cut_point['private'].cumsum()
    plt.plot(df_cut_point['nro_envios'][4000:20000], df_cut_point['private_cum'][4000:20000], label='Ganancia Privada Acumulada')
    max_private_cum = df_cut_point['private_cum'][4000:20000].max()
    max_private_idx = df_cut_point['private_cum'][4000:20000].idxmax()
    plt.axvline(x=max_private_idx, color='r', linestyle='--', label=f'Máximo Ganancia Privada en {max_private_idx}')

  plt.title('Curva de Ganancia Pública y Privada')
  plt.xlabel('Número de envíos')
  plt.ylabel('Ganancia Acumulada')
  plt.legend()
  plt.show()

analisis_2(semilla, 4000, 500, 10, private=False)

NameError: name 'ganancia' is not defined

sin prestamos_personales, con lags + delta_lag

## Tarea:

1. **Generar Dataset**  
   - Utilice las técnicas de *feature engineering* vistas en las clases anteriores para generar un nuevo conjunto de datos.
   
2. **Optimización de LightGBM (LGBM)**  
   - Ajuste el modelo de LightGBM utilizando una mayor cantidad de árboles y realice una exploración más exhaustiva de los hiperparámetros para mejorar su rendimiento.
   
3. **Incluir Nuevos Parámetros en la Optimización**  
   - Revise la documentación de los parámetros de LightGBM. Evalúe la inclusión de otros parámetros en el proceso de optimización, y ajuste el modelo con estos nuevos parámetros.
   
4. **Selección del Mejor Modelo**  
   - Entre los cinco mejores modelos obtenidos en cada optimización, seleccione el que considere más adecuado para la competencia en Kaggle.
   - Documente las pruebas que realizó para seleccionar el mejor modelo. Justifique su decisión con métricas relevantes y análisis comparativos.
5. Escriba y comparta por **Zulip** una función que envíe prepare el dataset que es necesario enviar a **kaggle** con los N clientes con mayor probabilidad.